In [1]:
from typing import Generator
import psycopg2


class PostgresStorage:

    def __init__(self, conn):
        self.conn = conn
        self.cursor = conn.cursor()

    @staticmethod
    def connect(host, port=5432, user='postgres', password='password', dbname='habr'):
        return PostgresStorage(conn=psycopg2.connect(
            host=host, port=port, user=user, password=password, dbname=dbname)
        )

    def get_posts(self, habs_list: list = None, tags_list: list = None, count = 0, ordered = True) -> Generator:
        if not habs_list and not tags_list:
            sql = 'SELECT * FROM posts'
            if ordered:
                sql = sql + ' LIMIT %d' % count if count > 0 else sql
            self.cursor.execute(sql)
        elif habs_list:
            self.__get_posts_by_habs(habs_list, count)
        elif tags_list:
            self.__get_posts_by_tags(tags_list, count)
        posts_gen = (post for post in self.cursor.fetchall())
        return posts_gen

    def get_posts_texts(self, habs_list: list = None, tags_list: list = None, count=0) -> Generator:
        posts_texts_gen = (post[2] for post in self.get_posts(habs_list, tags_list, count))
        return posts_texts_gen

    def __get_posts_by_habs(self, habs_list: list, count=0) -> None:
        sql = '''SELECT P.* 
                   FROM posts P JOIN habs H ON P.post_id = H.post_id
                  WHERE H.hab in (%s)''' % ''.join(["'" + str(hab) + "', " for hab in habs_list])[:-2]
        sql = sql + " LIMIT %d" % count if count > 0 else sql
        self.cursor.execute(sql)

    def __get_posts_by_tags(self, tags_list: list, count=0) -> None:
        sql = '''SELECT P.* 
                   FROM posts P JOIN tags T ON P.post_id = T.post_id
                  WHERE T.tag in (%s)''' % ''.join(["'" + str(tag) + "', " for tag in tags_list])[:-2]
        sql = sql + " LIMIT %d" % count if count > 0 else sql
        self.cursor.execute(sql)


ModuleNotFoundError: No module named 'psycopg2'

In [2]:
import sys
!{sys.executable} -m pip install psycopg2

     |████████████████████████████████| 383 kB 1.6 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /home/vnkrtv/anaconda3/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-vt4_42li/psycopg2/setup.py'"'"'; __file__='"'"'/tmp/pip-install-vt4_42li/psycopg2/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-install-vt4_42li/psycopg2/pip-egg-info
         cwd: /tmp/pip-install-vt4_42li/psycopg2/
    Complete output (23 lines):
    running egg_info
    creating /tmp/pip-install-vt4_42li/psycopg2/pip-egg-info/psycopg2.egg-info
    writing /tmp/pip-install-vt4_42li/psycopg2/pip-egg-info/psycopg2.egg-info/PKG-INFO
    writing dependency_links to /tmp/pip-install-vt4_42li/psycopg2/pip-egg-info/psycopg2.egg-info/dependency_links.txt
    writing top-level names to /tmp/pip-instal

In [3]:
pg_text_gen = PostgresStorage.connect('172.17.0.2')
text_gen = pg_text_gen.get_posts_texts(count=20, habs_list=['Математика'])

In [5]:
text_corpus = list(text_gen)

In [6]:
len(text_corpus)

20

In [7]:
conn = psycopg2.connect(host='172.17.0.3', port=5432, user='postgres', password='password', dbname='postgres')
cur = conn.cursor()

In [8]:
%%time
pg_proc = 'CALL train_chain(%s::text[],%s::integer)'
state_size = 3
cur.execute(pg_proc, [text_corpus, state_size])
conn.commit()

CPU times: user 19 ms, sys: 555 µs, total: 19.6 ms
Wall time: 1min 12s


In [ ]:
query = 'SELECT '
cur.execute(pg_proc, [text_corpus, state_size])